In [ ]:
# -*- coding: utf-8 -*-
"""voicegpt.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1qY4G249zGOyd3UGP3bbplXTw-rjMeBEo
"""

! pip install gradio openai gtts pydub numpy requests groq openai-whisper
!apt-get install -y ffmpeg

import os #intract with the operating system

import gradio as gr # Used for creating web-based user interfaces for machine learning apps

import whisper # OpenAI's speech recognition model for transcribing audio to text

from gtts import gTTS # Google Text-to-Speech API to convert text into spoken audio

from IPython.display import Audio # For playing audio directly in Jupyter Notebooks

from groq import Groq # Likely a client for interacting with Groq's AI APIs (alternative to OpenAI)

import openai # OpenAI Python library to interact with GPT models and other OpenAI services

import numpy as np # Numerical computing library used for manipulating audio data and arrays


import os # Provides tools for interacting with the operating system

import gradio as gr # Used to create interactive web UIs for machine learning and AI applications

import whisper # OpenAI’s automatic speech recognition (ASR) model to convert speech to text

from gtts import gTTS # Google Text-to-Speech: converts text into spoken audio using Google's TTS engine

from groq import Groq, GroqError # Groq: client for accessing Groq's AI APIs; GroqError handles API-related exceptions

from typing import Tuple, Union # Provides type hints for better code clarity and editor support (e.g., function return types)

# Initialize Whisper model
model = whisper.load_model("base")

# Initialize Groq API client with the API key directly
api_key = "groq_api_key"
try:
    client = Groq(api_key=api_key)
except Exception as e:
    raise RuntimeError(f"Failed to initialize Groq client: {e}")

def transcribe_and_respond(audio: str) -> Tuple[str, Union[str, None]]:
    try:
        # Step 1: Transcribe the audio using Whisper
        transcription = model.transcribe(audio)
        user_input = transcription['text']

        # Step 2: Generate a response using Groq API and LLaMA model
        try:
            chat_completion = client.chat.completions.create(
                messages=[{"role": "user", "content": user_input}],
                model="llama3-8b-8192",
            )
            response_text = chat_completion.choices[0].message.content
        except GroqError as e:
            return f"Error in Groq API call: {e}", None

        # Step 3: Convert the response text to speech using gTTS
        tts = gTTS(response_text)
        audio_path = "response.mp3"
        tts.save(audio_path)

        return response_text, audio_path

    except FileNotFoundError:
        return "Error: Audio file not found.", None
    except whisper.WhisperError as e:
        return f"Error in transcription: {e}", None
    except Exception as e:
        return f"An unexpected error occurred: {e}", None

# Gradio interface for real-time interaction
interface = gr.Interface(
    fn=transcribe_and_respond,
    inputs=gr.Audio(type="filepath"),
    outputs=[gr.Textbox(label="Response"), gr.Audio(label="Voice Response")],
    live=True
)

interface.launch()